In [0]:
from delta.tables import DeltaTable
from pyspark.sql import functions as F

# Reference existing Delta table by path
delta_path = "dbfs:/Volumes/workspace/default/kaggle_volume/delta/events"
deltaTable = DeltaTable.forPath(spark, delta_path)


In [0]:
# Create incremental data
updates = (
    spark.read.format("delta").load(delta_path)
    .sample(0.02, seed=42)          # small batch
    .withColumn("price", F.col("price") + 5)  # simulate change
)


In [0]:
# Deduplicate source
updates_clean = updates.dropDuplicates(
    ["user_id", "event_time", "product_id"]
)

In [0]:
# Merge
deltaTable.alias("t").merge(
    updates_clean.alias("s"),
    """
    t.user_id = s.user_id
    AND t.event_time = s.event_time
    AND t.product_id = s.product_id
    """
).whenMatchedUpdateAll() \
 .whenNotMatchedInsertAll() \
 .execute()

print("Incremental MERGE completed")


Incremental MERGE completed


In [0]:

%sql
-- Query Historical Versions (Time Travel)
-- step1- View table history
DESCRIBE HISTORY workspace.default.events_table;



version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
0,2026-01-12T05:37:49.000Z,72099111605464,sonali.gupta@vensysco.in,CREATE OR REPLACE TABLE AS SELECT,"Map(partitionBy -> [], clusterBy -> [], description -> null, isManaged -> true, properties -> {""delta.enableDeletionVectors"":""true""}, statsOnLoad -> true)",null,List(1556992834470568),0112-050619-8lqz2v4h-v2n,null,WriteSerializable,false,"Map(numFiles -> 43, numRemovedFiles -> 0, numRemovedBytes -> 0, numDeletionVectorsRemoved -> 0, numOutputRows -> 42448764, numOutputBytes -> 1405244778)",null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13


In [0]:
# step2- Read an old version
v0 = spark.read.format("delta") \
    .option("versionAsOf", 0) \
    .load(delta_path)

print("Version 0 rows:", v0.count())

Version 0 rows: 42448764


In [0]:
# step3- Read by timestamp
old_data = spark.read.format("delta") \
    .option("timestampAsOf", "2024-01-01") \
    .load(delta_path)

old_data.show(5)


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-7382243017409246>, line 6
      1 # step3- Read by timestamp
      2 old_data = spark.read.format("delta") \
      3     .option("timestampAsOf", "2024-01-01") \
      4     .load(delta_path)
----> 6 old_data.show(5)

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/dataframe.py:1123, in DataFrame.show(self, n, truncate, vertical)
   1122 def show(self, n: int = 20, truncate: Union[bool, int] = True, vertical: bool = False) -> None:
-> 1123     print(self._show_string(n, truncate, vertical))

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/dataframe.py:876, in DataFrame._show_string(self, n, truncate, vertical)
    859     except ValueError:
    860         raise PySparkTypeError(
    861             errorClass="NOT_BOOL",
    862             messageParameters={
   (...)
   

In [0]:

%sql
-- OPTIMIZE & ZORDER
OPTIMIZE workspace.default.events_table
ZORDER BY (event_type, user_id);


path,metrics
,"List(26, 43, List(30174253, 56176753, 4.4598923115384616E7, 26, 1159572001), List(5827713, 35958801, 3.268011111627907E7, 43, 1405244778), 0, List(minCubeSize(107374182400), List(0, 0), List(43, 1405244778), 0, List(43, 1405244778), 1, null), null, 0, 1, 43, 0, false, 0, 0, 1768284401303, 1768284420459, 8, 1, null, List(0, 0), null, 9, 9, 64142, 0, null)"


In [0]:

%sql
-- Clean Old Files (VACUUM)
VACUUM workspace.default.events_table RETAIN 168 HOURS;


path
""
